In [70]:
from DistanceHelper.DistanceDetector import detectPeople
from DistanceHelper.DistanceDetector import MIN_CONF
from DistanceHelper.DistanceDetector import MIN_DISTANCE
from DistanceHelper.DistanceDetector import NMS_THRESH
from tqdm.notebook import tqdm
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import time

In [71]:
videoFile = r'videos/testVideo_Trim.mp4'
configPath = r'YOLO_DB/yolov3.cfg'
weightsPath = r'YOLO_DB/yolov3.weights'
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

## Operation Selector

In [72]:
previewWhileProcess = True
saveLocally = False

In [73]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

['yolo_82', 'yolo_94', 'yolo_106']


In [74]:
cam = cv2.VideoCapture(videoFile)
fps = int(cam.get(cv2.CAP_PROP_FPS))
totalFPS = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))
computedFps = 0
print(fps, totalFPS)

30 149


Saving properties

In [75]:
size = (int(cam.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print(size)
fourcc =cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('videos/outputX.mp4', fourcc, 10, size, True) 

(1920, 1080)


In [76]:
LABELS = []
with open("YOLO_DB/coco.names", 'rt') as f:
    LABELS = f.read().rstrip("\n").split('\n')
print(len(LABELS))
print(LABELS.index("person"))

80
0


In [77]:
color_violation = (0,0, 255)
color_safe = (0,255,0)

In [78]:
pbar = tqdm(total=totalFPS, desc="Progress") #progressBar

Progress:   0%|          | 0/149 [00:00<?, ?it/s]

In [79]:
print("[INFO] init Process..")
while True:
    success, frame = cam.read()
    
    if not success:
        print("[INFO] EOF Process..")
        break
        
    if not saveLocally:
        frame = imutils.resize(frame, width=700)
    results = detectPeople(frame, net, ln, personIdx=LABELS.index('person'))


    voilation = set()

    if len(results) >=2:
        centroids = np.array([r[2] for r in results])
        distanceMap = dist.cdist(centroids, centroids, metric = "euclidean")

#   print(distanceMap.shape)
        for i in range(distanceMap.shape[0]):
            for j in range(i+1, distanceMap.shape[1]):

                #if distance between 2person is less minDistance
                if distanceMap[i, j] < MIN_DISTANCE:
                    voilation.add(i)
                    voilation.add(j)


        #Loop over results and make visualizations
    for (i, (prob, bbox, centroid)) in enumerate(results):
        (startX, startY, endX, endY)  = bbox
        (cX, cY) = centroid

        color = color_safe

        if i in voilation:
            color = color_violation

        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.circle(frame, (cX, cY), 5, color, 1)
    
    #should be display
    if previewWhileProcess:
        cv2.imshow("Social Distance", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
    
    if saveLocally:
        writer.write(frame)
        
    # 
    computedFps +=1
    pbar.update(1)

pbar.close()
writer.release()     
cv2.destroyAllWindows()


[INFO] init Process..
[INFO] EOF Process..
